In [298]:
import pandas as pd
import numpy as np
data = pd.read_csv('polarity_labeling.csv')
data.head()

C:\Users\tjlee\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,asin,date,product,review,score,user,CS_SS,target,feature,polarity
0,B00000JFOA,"01 6, 2000",c-2000,the newer model c-2020 has all the fixes exce...,4.0,Lee,CS,c-2020,lens,0
1,B000023VQC,"06 24, 2001",pdc-700,it likes to change from xga to vga etc.. *the ...,2.0,A. Gray,CS,vga,battery,0
2,B00003OPEY,"04 26, 2000",dsc-f505,the included memory stick is a little small b...,5.0,David Pirmann,CS,110,memory,1
3,B00004R8V6,"09 5, 2000",d-360l,it also adds a nifty little optical function ...,5.0,"Baltic Books ""Vic""",CS,d-460,"optical,battery,lcd,compression,flash",1
4,B00004R8VC,"04 18, 2000",d-460,i read a lot of reviews on the d450 optical be...,5.0,JR,CS,d450,optical,-1


In [302]:
class matrix:
    import pandas as pd
    #생성자
    def __init__(self, df, feature): #매개값은 전 처리 데이터, 매트릭스 만들려는 feature
        df = df[df.feature.str.contains(feature)] #해당 feature 리뷰 추출
        self.product_list = df['product'].unique() #product list
        df_pol = df[df['polarity'] !=0 ] #리뷰가 중립인 값 제거
        self.df_cs = df_pol[df_pol['CS_SS']=='CS'] #CS DF
        self.df_ss = df_pol[df_pol['CS_SS']=='SS'] #SS DF
        self.matrix_df = pd.DataFrame(float(0), index=self.product_list, columns = self.product_list) #매트릭스 생성
    
    #SS 계산 관리 함수
    def ss_calc(self, Gby):
        #SS 계산 함수(groupby 계산 전용)
        def calc(series):
            #SS 계산
            val_cnt = series.value_counts()
            if len(val_cnt.index) == 2:
                return val_cnt[1] / val_cnt[-1]
            elif 1 not in val_cnt.index:
                return 0
            else:
                return val_cnt[1]
        
        result = Gby.agg(calc)
        return result
    #CS 계산 관리 함수
    def cs_calc(self, Gby):
        def calc(t, p):
            
            #여러개의 모델을 flatten 처리. 1차원 배열로 변경 한다
            p_ls = list(p)
            tmp_p = []
            tmp_t = []
            for idx, token in enumerate(t):
                if len(token.split(',')) > 1:
                    tmp_t.extend(token.split(','))
                    for i in range(len(token.split(','))):
                        tmp_p.append(p_ls[idx])
                else:
                    tmp_t.append(token)
                    tmp_p.append(p_ls[idx])
            
            #감성 분류 과정1 : 긍정 라벨 갯수와 부정 라벨 갯수를 샌다
            tmp_dict = dict(zip(tmp_t, [[0,0] for i in range(len(tmp_t))]))
            for idx, token in enumerate(tmp_t):
                if tmp_p[idx] == 1:
                    tmp_dict[token][0] += 1
                else:
                    tmp_dict[token][1] += 1
                    
            #감성 분류 과정2 : 가중치 계산 단계
            for k in tmp_dict:
                if tmp_dict[k][0] == 0:
                    tmp_dict[k] = 0
                elif tmp_dict[k][1] == 0:
                    tmp_dict[k] = tmp_dict[k][0]
                else:
                    tmp_dict[k] = tmp_dict[k][0] / tmp_dict[k][1]
                    
            #각 제품 에게 가중치를 주는 제품 리스트와 가중치 값 리스트를 반환
            return list(tmp_dict.keys()), list(tmp_dict.values())
        
        result = Gby.agg(lambda x:calc(x['target'],x['polarity']))
        
        #전처리 과정. 피벗 테이블로 이루어진 데이터프레임(칼럼이 2차원 구조)을 1차원으로 flatten
        result = result.reset_index()
        result.index = list(result['product'])
        result = result.drop('product', axis = 1)
        return result
    
    #이 함수를 사용해야 매트릭스가 생성됩니다.
    def make_matrix(self):
        #SS 가중치를 matrix에 삽입
        def ss_matrix(series):
            try:
                series[series.name]= self.ss_result[series.name]
                return series
            except:
                return series
            
        #CS 가중치를 matrix에 삽입    
        def cs_matrix(series):
            if series.name not in list(self.cs_result.index):
                return series
            
            mnw = self.cs_result.loc[series.name, ['target','polarity']]
            
            for idx, m in enumerate(mnw['target']):
                if m in list(series.index):
                    series[m] = mnw['polarity'][idx]
                    
            return series
        
        #SS 가중치 계산, 매트릭스 입력
        self.ss_result = self.ss_calc(self.df_ss['polarity'].groupby(self.df_ss['product']))
        self.matrix_df = self.matrix_df.apply(ss_matrix, axis=1)
        
        #CS 가중치 계산, 매트릭스 입력
        self.cs_result = self.cs_calc(self.df_cs[['target','polarity']].groupby(self.df_cs['product']))
        self.matrix_df = self.matrix_df.apply(cs_matrix, axis=1)
        
        #sum 행 포함
        self.matrix_df.loc['sum'] = self.matrix_df.apply(lambda x:sum(x))
        return self.matrix_df
    
    #매트릭스를 읽습니다
    def call_matrix(self):
        return self.matrix_df

In [304]:
feature = 'pixel' #원하는 feature를 입력하여 matrix를 생성
pixel_matrix = matrix(data2,feature) 
pixel_matrix.make_matrix().head()

,d-460,qv3000ex,s10,pdr-m60,c-3000,s100,mvc-fd95,40i,d-490,d30,...,p7800,aw1,ek-gn120zkaxar,l27,sx600,l830,wb35f,elph-150,g1x,hx400v/b
d-460,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
qv3000ex,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s10,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pdr-m60,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
c-3000,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [305]:
def Make_Rank(df, d = 0.85, MAX_ITERATIONS = 50, threshold = 0.0001):
    # sum = 0 인 열 전체를 지우는 함수
    df = df.T
    df=df[df['sum'] != 0].T
    
    product_list = df.columns
    
    # 초기 스코어 설정( 1 )
    score = np.ones(len(product_list))
    
    # 최대 반복 횟수 50 설정
    for iter in range(0,MAX_ITERATIONS):
        prev_score = np.copy(score)
        
        for i, p in enumerate(product_list):
            # 넘파이 배열의 사칙연산을 사용하여 for문을 줄임
            score[i] = (1-d) + d*sum(df.loc[p] / df.loc['sum'] * prev_score)

        # 반복 중 수치 변화가 일정 값 미만으로 떨어지면 코드 종료
        if np.sum(np.fabs(prev_score-score)) <= threshold:
            print('요기서 끝 :', iter)
            break
            
    normal_score = score / np.sqrt((np.sum(score**2)))
    dff = pd.DataFrame(normal_score, index = product_list)
    return dff

In [306]:
pagerank=Make_Rank(pixel_matrix.call_matrix())

요기서 끝 : 45


In [307]:
pagerank.columns = [feature]

In [155]:
pagerank.to_csv(f'{feature}_pagerank.csv',encoding = 'utf-8')